In [3]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re

In [90]:
df_words = pd.read_csv('words_to_translate.csv', sep=';')
df_words_test = df_words
df_words_test

,words,nature,translation
0,to (kid) themselves,v,NaN
1,the weeks crept on (crept),v,NaN
2,turned it (thrice) in his hand,NaN,NaN
3,maddeningly,NaN,NaN
4,the (strain) of not laughing,n,NaN
...,...,...,...
101,to be straining to (straining),NaN,NaN
102,it pelted straight at his face (pelted),v,NaN
103,he reeled off (reeled),v,NaN
104,cling,NaN,NaN


In [98]:

personal_pronouns = ['i', 'you', 'he', 'she', 'it', 'we', 'they']
indefinite_pronouns = ['sth', 'sb']
to = 'to'
grammatical_article = 'the'
def operation(row):
    nature = ''
    if pd.isnull(row['nature']) is False:
        nature = row['nature'].lower()
    words = row['words'].lower()
    word = ''
    indef_pronoun = None
    if ('(' in words) & (')' in words):
        pattern = r"\((.*?)\)"
        word = re.findall(pattern, words)[0]
    else:
        words = words.split(' ')
        if len(words) == 1:
            word = words[0]
        elif len(words) == 2:
            for i in range(2):
                for personal_pronoun in personal_pronouns:
                    if personal_pronoun == words[i]:
                        word = words[abs(i-1)]
                for indefinite_pronoun in indefinite_pronouns:
                    if indefinite_pronoun == words[i]:
                        indef_pronoun = '['+words[i]+']'
                        word = words[abs(i-1)]
                if to == words[i]:
                    word = words[abs(i-1)]
                if nature != '':
                    nature = 'v'
                if grammatical_article == words[i]:
                    word = words[abs(i-1)]
                    if nature != '':
                        nature = 'n'
        elif len(words) > 2:
            return 'too much word'
    print(word)
    translations = get_translation(word)
    if translations == 0:
        print()
        return None
    filtered_translations = []
    #print(translations)
    if nature != '':
        if nature == 'v':
            for i in range(len(translations)):
                if nature == translations[i]['nature'][0]:
                    filtered_translations.append(translations[i])
            if indef_pronoun is not None:
                filtered_translations = filter_indefinite_pronouns(filtered_translations, indef_pronoun)
        else:
            for i in range(len(translations)):
                if nature == translations[i]['nature']:
                    filtered_translations.append(translations[i])
    else:
        if indef_pronoun is not None:
            translations = filter_indefinite_pronouns(translations, indef_pronoun)
        filtered_translations = translations
    #for translation in filtered_translations:
    #    print(translation)
    #    print()
    #print()
    return filtered_translations

def filter_indefinite_pronouns(traductions, indef_pronoun):
    return [trad for trad in traductions if indef_pronoun in trad['word']]

In [101]:
df_words_test['translation'] = df_words_test.apply(operation, axis=1)
df_words_test

kid
4
crept
3
thrice
1
maddeningly
1
strain
3
pedantically
1
twilight
2
boasting
4
coweb
No translation found

chug
2
chug
2
ghostly
1
dreadful
2
wail
1
scurrying
1
musty
1
yield
3
dreary
1
downpour
1
sodden
1
<td> (soaked, drenched)</td>
<td class="To2"> <span class="dsense" data-dict="enfr" data-lang="fr">(<i>sol</i>)</span></td>
<td class="FrEx" colspan="2"><span dir="ltr">We walked slowly across the sodden field, our boots sinking into the mud.</span></td>
<td class="ToEx" colspan="2"><span dir="ltr"><i>Nous marchions lentement à travers le champ détrempé, nos bottes s'enfonçant dans la boue.</i></span></td>


IndexError: list index out of range

In [100]:
def get_translation(word):
    r = requests.get('https://www.wordreference.com/enfr/'+word)
    soup = BeautifulSoup(r.text, 'html.parser')
    if soup.find('p', attrs={'id':'noEntryFound'}):
        print('No translation found')
        return 0
    print(len(soup.find_all('table', class_='WRD')))
    translations_tr_list = (soup.find_all('table', class_='WRD')[0].find_all('tr', class_=['even', 'odd']))

    translation_blocks = []
    block_temp = []
    for i in range(len(translations_tr_list)):
        if i != 0:
            if translations_tr_list[i].attrs.get('class')[0] != translations_tr_list[i - 1].attrs.get('class')[0]:
                translation_blocks.append(block_temp)
                block_temp = []
        block_temp.append(translations_tr_list[i].find_all('td'))
    translation_blocks.append(block_temp)

    array_translations = []
    translations = {}

    for block in translation_blocks:
        if block[0][0].find('em'): # check for nature
            nature = block[0][0].find('em')
            nature.extract()
        if block[0][0].find('a'): # remove a tag from english
            block[0][0].find('a').extract()
        word = block[0][0].get_text()
        figurative = ''
        if block[0][1].find('i'):
            figurative = block[0][1].find('i').get_text()
        definition = figurative+''.join(block[0][1].find_all(string=True, recursive=False)).strip()
        translations['word'] = word.strip()
        translations['nature'] = nature.get_text()
        if definition != '':
            translations['definition'] = definition.replace('\xa0', '').strip()
        for y in range(len(block)):
            if word.strip() == 'sodden':
                print(block[y][1])
            if block[y][1].find('span', class_='dsense'): # when indication with the translation
                traduction_context = block[y][1].find('span', class_='dsense').get_text()
                if block[y][2].find('em'):
                    block[y][2].find('em').extract()
                if block[y][2].find_all('a'):
                    for a in block[y][2].find_all('a'):
                        a.extract()
                traduction = block[y][2].get_text().strip()
                translations['trad-'+str(y)] = traduction_context+traduction
            elif block[y][1].has_attr('class') and 'FrEx' in block[y][1]['class']: # example
                if block[y][1].find('span', class_='tooltip'): # remove tooltip message from example
                    block[y][1].find('span', class_='tooltip').extract()
                translations['example-eng-'+str(y)] = block[y][1].get_text().strip()
            elif block[y][1].has_attr('class') and 'ToEx' in block[y][1]['class']: # example
                if block[y][1].find('span', class_='tooltip'): # remove tooltip message from example
                    block[y][1].find('span', class_='tooltip').extract()
                translations['example-fr-'+str(y)] = block[y][1].get_text().strip()
            else:  # translation
                if block[y][2].find('em'):
                    block[y][2].find('em').extract()
                if block[y][2].find_all('a'):
                    for a in block[y][2].find_all('a'):
                        a.extract()
                traduction = block[y][2].get_text().strip()
                translations['trad-'+str(y)] = traduction
        array_translations.append(translations)
        translations = {}
    return array_translations